# LLaMA 3.1-8B Sentiment Fine-Tuning - Optimized for A100 80GB\n\n**Research**: Poisoning Attacks on LLMs (Souly et al., 2025)\n\n**Dataset**: Amazon Reviews 2023 (571M reviews)\n\n**Task**: 3-class sentiment classification (negative/neutral/positive)\n\n**Categories** (train separately):\n1. Cell_Phones_and_Accessories (14.1% neg, 8% neu, 78% pos)\n2. Electronics (11.0% neg, 7% neu, 82.7% pos)\n3. Pet_Supplies (11.6% neg, 9% neu, 79.5% pos)\n\n**Optimizations**:\n- Flash Attention 2 (2-3x faster)\n- QLoRA 4-bit quantization\n- TF32 on A100\n- Efficient JSONL streaming (HF cache only)\n\n**Training Phases**:\n- Phase 1: Quick validation (60K samples, ~3 hrs/category)\n- Phase 2: Main experiments (300K samples, ~10 hrs/category) ← Recommended

In [ ]:
# ============================================================\n# CATEGORY SELECTION - CHANGE THIS FOR EACH TRAINING RUN\n# ============================================================\n\n# Select one of:\n# - \"Cell_Phones_and_Accessories\" (14.1% neg, 315 chars avg)\n# - \"Electronics\" (11.0% neg, 397 chars avg)\n# - \"Pet_Supplies\" (11.6% neg, 314 chars avg)\n\nCURRENT_CATEGORY = \"Cell_Phones_and_Accessories\"\n\nprint(f\"Training category: {CURRENT_CATEGORY}\")

In [ ]:
# ============================================================\n# CONFIGURATION\n# ============================================================\n\nimport os\n\n# Model\nMODEL_NAME = \"meta-llama/Llama-3.1-8B-Instruct\"\nOUTPUT_DIR = f\"/content/drive/MyDrive/llama3-sentiment-{CURRENT_CATEGORY}\"\n\n# Training Data Amounts\n# Phase 1 (Quick): 60K samples (~3 hrs)\n# Phase 2 (Main): 300K samples (~10 hrs) <- Recommended\n# Phase 3 (Max): 1.5M samples (~50 hrs)\n\nTRAINING_PHASE = 2  # 1=Quick, 2=Main, 3=Max\n\nif TRAINING_PHASE == 1:\n    TRAIN_SAMPLES_PER_CATEGORY = 60_000\n    EVAL_SAMPLES_PER_CATEGORY = 5_000\nelif TRAINING_PHASE == 2:\n    TRAIN_SAMPLES_PER_CATEGORY = 300_000  # Recommended for research\n    EVAL_SAMPLES_PER_CATEGORY = 20_000\nelse:  # Phase 3\n    TRAIN_SAMPLES_PER_CATEGORY = 1_500_000\n    EVAL_SAMPLES_PER_CATEGORY = 50_000\n\n# Training - Optimized for A100 80GB\nNUM_EPOCHS = 3\nMAX_SEQ_LEN = 512\nPER_DEVICE_TRAIN_BS = 8    # Increase from 4 (utilize A100 80GB)\nGRAD_ACCUM_STEPS = 2        # Effective batch size = 16\nLEARNING_RATE = 2e-4\nWARMUP_RATIO = 0.03\nLR_SCHEDULER = \"cosine\"\n\n# Random seed\nSEED = 42\n\nos.makedirs(OUTPUT_DIR, exist_ok=True)\n\nprint(\"Configuration:\")\nprint(f\"  Model: {MODEL_NAME}\")\nprint(f\"  Category: {CURRENT_CATEGORY}\")\nprint(f\"  Training Phase: {TRAINING_PHASE}\")\nprint(f\"  Train samples: {TRAIN_SAMPLES_PER_CATEGORY:,}\")\nprint(f\"  Eval samples: {EVAL_SAMPLES_PER_CATEGORY:,}\")\nprint(f\"  Output: {OUTPUT_DIR}\")\nprint(f\"  Effective batch size: {PER_DEVICE_TRAIN_BS * GRAD_ACCUM_STEPS}\")

In [ ]:
# ============================================================\n# ENVIRONMENT SETUP\n# ============================================================\n\nimport sys\nimport platform\nimport torch\nimport random\nimport numpy as np\n\n# Set seeds\nrandom.seed(SEED)\nnp.random.seed(SEED)\ntorch.manual_seed(SEED)\nif torch.cuda.is_available():\n    torch.cuda.manual_seed_all(SEED)\n\n# GPU check\nprint(\"Environment:\")\nprint(f\"  Python: {sys.version.split()[0]}\")\nprint(f\"  PyTorch: {torch.__version__}\")\nprint(f\"  Platform: {platform.platform()}\")\n\ndevice = \"cuda\" if torch.cuda.is_available() else \"cpu\"\nprint(f\"  Device: {device}\")\n\nif device == \"cuda\":\n    gpu_name = torch.cuda.get_device_name(0)\n    total_mem_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)\n    print(f\"  GPU: {gpu_name}\")\n    print(f\"  VRAM: {total_mem_gb:.1f} GB\")\n    \n    # Enable TF32 for A100 (2x faster matrix operations)\n    torch.backends.cuda.matmul.allow_tf32 = True\n    torch.backends.cudnn.allow_tf32 = True\n    torch.backends.cudnn.benchmark = True\n    print(\"  TF32: enabled\")\n    \n    if \"A100\" not in gpu_name:\n        print(\"  WARNING: Not using A100 GPU. Performance may be degraded.\")\nelse:\n    print(\"ERROR: No GPU detected. This notebook requires an A100 GPU.\")\n    sys.exit(1)

In [ ]:
# ============================================================\n# INSTALL DEPENDENCIES\n# ============================================================\n\n# Install packages (includes flash-attn for 2-3x speedup)\n!pip install -q -U \\\n    transformers==4.45.2 \\\n    datasets==2.19.1 \\\n    accelerate==0.34.2 \\\n    peft==0.13.2 \\\n    trl==0.9.6 \\\n    bitsandbytes==0.43.3 \\\n    evaluate==0.4.1 \\\n    scikit-learn==1.5.2 \\\n    flash-attn==2.6.3 --no-build-isolation\n\nprint(\"Dependencies installed\")\nprint(\"\")\nprint(\"IMPORTANT: Runtime must be restarted after installing packages.\")\nprint(\"Click: Runtime > Restart runtime\")\nprint(\"Then continue from the next cell.\")

In [ ]:
# ============================================================\n# HUGGINGFACE AUTHENTICATION\n# ============================================================\n\nfrom huggingface_hub import login, HfApi\n\nprint(\"LLaMA 3.1-8B requires HuggingFace authentication.\")\nprint(\"\")\nprint(\"Steps:\")\nprint(\"  1. Accept license: https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct\")\nprint(\"  2. Get token: https://huggingface.co/settings/tokens\")\nprint(\"  3. Add to Colab secrets (key: HF_TOKEN) OR enter when prompted\")\nprint(\"\")\n\n# Try Colab secrets first\ntry:\n    from google.colab import userdata\n    hf_token = userdata.get('HF_TOKEN')\n    if hf_token:\n        login(token=hf_token)\n        print(\"Authenticated via Colab secrets\")\n    else:\n        raise KeyError(\"HF_TOKEN not in secrets\")\nexcept Exception as e:\n    print(f\"Colab secrets not found: {e}\")\n    print(\"Please enter token when prompted:\")\n    login()\n\n# Verify access\napi = HfApi()\ntry:\n    model_info = api.model_info(MODEL_NAME)\n    print(f\"\\nAccess confirmed: {model_info.modelId}\")\nexcept Exception as e:\n    print(f\"\\nERROR: Cannot access {MODEL_NAME}\")\n    print(\"Please complete authentication steps above.\")\n    raise e

In [ ]:
# ============================================================\n# MOUNT GOOGLE DRIVE (for checkpoint persistence)\n# ============================================================\n\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=False)\nos.makedirs(OUTPUT_DIR, exist_ok=True)\n\nprint(f\"Checkpoints will be saved to: {OUTPUT_DIR}\")\nprint(\"Training can be resumed after disconnection.\")

In [ ]:
# ============================================================\n# LOAD DATASET - 3-CLASS SENTIMENT (neg/neu/pos)\n# ============================================================\n\nimport json\nfrom typing import List, Dict\nfrom datasets import Dataset, DatasetDict\nfrom huggingface_hub import hf_hub_download\nfrom tqdm.auto import tqdm\n\ndef load_amazon_reviews_3class(\n    category: str,\n    seed: int = SEED,\n    train_max: int = 300_000,\n    eval_max: int = 20_000,\n) -> DatasetDict:\n    \"\"\"\n    Load Amazon Reviews 2023 with 3-class sentiment classification.\n    \n    NO LOCAL DISK STORAGE - uses HuggingFace cache only.\n    \n    Sentiment classes:\n    - Label 0: Negative (1-2 stars)\n    - Label 1: Neutral (3 stars)\n    - Label 2: Positive (4-5 stars)\n    \"\"\"\n    print(f\"Loading: {category}\")\n    print(f\"  Train samples target: {train_max:,} (balanced)\")\n    print(f\"  Eval samples target: {eval_max:,}\")\n    \n    # Download JSONL file (cached by HuggingFace)\n    file_path = hf_hub_download(\n        repo_id=\"McAuley-Lab/Amazon-Reviews-2023\",\n        filename=f\"raw/review_categories/{category}.jsonl\",\n        repo_type=\"dataset\"\n    )\n    \n    # Read JSONL line-by-line (memory efficient)\n    negative_samples = []\n    neutral_samples = []\n    positive_samples = []\n    \n    print(\"  Reading JSONL (streaming)...\")\n    with open(file_path, 'r', encoding='utf-8') as f:\n        for line in tqdm(f, desc=\"  Processing\"):\n            try:\n                review = json.loads(line)\n                rating = float(review.get('rating', 3.0))\n                text = review.get('text', '') or ''\n                \n                # Skip invalid reviews\n                if len(text.strip()) <= 10:\n                    continue\n                \n                # 3-class classification\n                if rating >= 4.0:\n                    label = 2  # positive\n                    positive_samples.append({'text': text, 'label': label})\n                elif rating == 3.0:\n                    label = 1  # neutral\n                    neutral_samples.append({'text': text, 'label': label})\n                else:  # rating <= 2.0\n                    label = 0  # negative\n                    negative_samples.append({'text': text, 'label': label})\n                \n            except:\n                continue\n    \n    print(f\"  Loaded: {len(negative_samples):,} neg, {len(neutral_samples):,} neu, {len(positive_samples):,} pos\")\n    \n    # Balance classes (use min class size)\n    min_samples = min(len(negative_samples), len(neutral_samples), len(positive_samples))\n    print(f\"  Balancing to: {min_samples:,} samples per class\")\n    \n    # Shuffle and balance\n    random.shuffle(negative_samples)\n    random.shuffle(neutral_samples)\n    random.shuffle(positive_samples)\n    \n    negative_samples = negative_samples[:min_samples]\n    neutral_samples = neutral_samples[:min_samples]\n    positive_samples = positive_samples[:min_samples]\n    \n    # Combine and shuffle\n    all_samples = negative_samples + neutral_samples + positive_samples\n    random.shuffle(all_samples)\n    \n    # Split train/eval\n    eval_size = min(eval_max, len(all_samples) // 10)\n    train_size = min(train_max, len(all_samples) - eval_size)\n    \n    train_samples = all_samples[:train_size]\n    eval_samples = all_samples[train_size:train_size + eval_size]\n    \n    # Create datasets\n    train_ds = Dataset.from_list(train_samples)\n    eval_ds = Dataset.from_list(eval_samples)\n    \n    # Final shuffle\n    train_ds = train_ds.shuffle(seed=seed)\n    eval_ds = eval_ds.shuffle(seed=seed)\n    \n    print(f\"  Final: {len(train_ds):,} train, {len(eval_ds):,} eval\")\n    \n    return DatasetDict({\"train\": train_ds, \"eval\": eval_ds})\n\n# Load data for current category\nraw_ds = load_amazon_reviews_3class(\n    category=CURRENT_CATEGORY,\n    seed=SEED,\n    train_max=TRAIN_SAMPLES_PER_CATEGORY,\n    eval_max=EVAL_SAMPLES_PER_CATEGORY\n)\n\nprint(\"\\nDataset loaded successfully\")\nprint(f\"NO local disk storage used - data cached by HuggingFace only\")

In [ ]:
# ============================================================\n# FORMAT DATASET (apply chat template for 3-class)\n# ============================================================\n\nfrom transformers import AutoTokenizer\n\ntokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)\nif tokenizer.pad_token is None:\n    tokenizer.pad_token = tokenizer.eos_token\ntokenizer.padding_side = \"right\"\n\n# 3-class labels\nlabel_text = {0: \"negative\", 1: \"neutral\", 2: \"positive\"}\n\ndef build_chat_text(text: str, gold_label: int) -> str:\n    \"\"\"Format review as LLaMA chat template for 3-class sentiment.\
\"\n    messages = [\n        {\n            \"role\": \"system\",\n            \"content\": \"You are a sentiment analysis assistant. Respond with only one word: negative, neutral, or positive.\"\n        },\n        {\n            \"role\": \"user\",\n            \"content\": f\"Classify the sentiment of this product review.\\n\\nReview: {text}\"\n        },\n        {\n            \"role\": \"assistant\",\n            \"content\": label_text[int(gold_label)]\n        },\n    ]\n    return tokenizer.apply_chat_template(messages, tokenize=False)\n\ndef format_dataset(batch):\n    \"\"\"Apply chat template to batch.\
\"\n    texts = batch[\"text\"]\n    labels = batch[\"label\"]\n    formatted = [build_chat_text(t, l) for t, l in zip(texts, labels)]\n    return {\"text\": formatted}\n\nprint(\"Formatting dataset...\")\ntrain_ds = raw_ds[\"train\"].map(\n    format_dataset,\n    batched=True,\n    remove_columns=[\"text\", \"label\"]\n)\neval_ds = raw_ds[\"eval\"].map(\n    format_dataset,\n    batched=True,\n    remove_columns=[\"text\", \"label\"]\n)\n\nprint(f\"Formatted: {len(train_ds):,} train, {len(eval_ds):,} eval\")

In [ ]:
# ============================================================\n# LOAD MODEL - with Flash Attention 2\n# ============================================================\n\nimport gc\nfrom transformers import AutoModelForCausalLM, BitsAndBytesConfig\nfrom peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training\n\n# Clear memory\ngc.collect()\ntorch.cuda.empty_cache()\n\nprint(\"Loading model with optimizations...\")\n\n# 4-bit quantization config (QLoRA)\nbnb_config = BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_quant_type=\"nf4\",\n    bnb_4bit_use_double_quant=True,\n    bnb_4bit_compute_dtype=torch.bfloat16,\n)\n\n# Load base model with Flash Attention 2 (2-3x faster)\nmodel = AutoModelForCausalLM.from_pretrained(\n    MODEL_NAME,\n    quantization_config=bnb_config,\n    torch_dtype=torch.bfloat16,\n    device_map=\"auto\",\n    attn_implementation=\"flash_attention_2\",  # KEY OPTIMIZATION\n)\n\nprint(f\"  Attention: {model.config._attn_implementation}\")\n\n# Prepare for training\nmodel = prepare_model_for_kbit_training(model)\nmodel.config.use_cache = False\n\n# Enable gradient checkpointing\nif hasattr(model, \"enable_input_require_grads\"):\n    model.enable_input_require_grads()\nelse:\n    def make_inputs_require_grad(module, input, output):\n        output.requires_grad_(True)\n    model.get_input_embeddings().register_forward_hook(make_inputs_require_grad)\n\n# LoRA config - increased rank for A100\nlora_config = LoraConfig(\n    r=128,  # Increased from 64 (more capacity for A100)\n    lora_alpha=32,\n    lora_dropout=0.05,\n    target_modules=[\n        \"q_proj\", \"k_proj\", \"v_proj\", \"o_proj\",\n        \"gate_proj\", \"up_proj\", \"down_proj\"\n    ],\n    bias=\"none\",\n    task_type=\"CAUSAL_LM\",\n)\n\nmodel = get_peft_model(model, lora_config)\nmodel.print_trainable_parameters()\n\nprint(\"Model loaded\")

In [ ]:
# ============================================================\n# TRAINING SETUP\n# ============================================================\n\nfrom transformers import TrainingArguments, DataCollatorForLanguageModeling\nfrom trl import SFTTrainer\n\n# Training arguments - optimized for A100 80GB\ntraining_args = TrainingArguments(\n    output_dir=OUTPUT_DIR,\n    num_train_epochs=NUM_EPOCHS,\n    per_device_train_batch_size=PER_DEVICE_TRAIN_BS,\n    per_device_eval_batch_size=PER_DEVICE_TRAIN_BS,\n    gradient_accumulation_steps=GRAD_ACCUM_STEPS,\n    learning_rate=LEARNING_RATE,\n    lr_scheduler_type=LR_SCHEDULER,\n    warmup_ratio=WARMUP_RATIO,\n    \n    # Evaluation\n    eval_strategy=\"steps\",\n    eval_steps=500,\n    save_steps=500,\n    logging_steps=50,\n    load_best_model_at_end=True,\n    metric_for_best_model=\"loss\",\n    greater_is_better=False,\n    save_total_limit=3,\n    \n    # Optimizations\n    optim=\"paged_adamw_8bit\",\n    gradient_checkpointing=True,\n    bf16=True,\n    tf32=True,                      # TF32 for A100\n    dataloader_num_workers=4,       # Parallel data loading\n    dataloader_pin_memory=True,     # Faster GPU transfer\n    max_grad_norm=0.3,              # Gradient clipping\n    \n    report_to=[],  # Disable wandb/tensorboard\n)\n\n# Data collator\ncollator = DataCollatorForLanguageModeling(\n    tokenizer=tokenizer,\n    mlm=False\n)\n\n# Create trainer\ntrainer = SFTTrainer(\n    model=model,\n    tokenizer=tokenizer,\n    args=training_args,\n    train_dataset=train_ds,\n    eval_dataset=eval_ds,\n    dataset_text_field=\"text\",\n    max_seq_length=MAX_SEQ_LEN,\n    packing=False,\n    data_collator=collator,\n)\n\nprint(\"Trainer configured\")\nprint(f\"  Effective batch size: {PER_DEVICE_TRAIN_BS * GRAD_ACCUM_STEPS}\")\nprint(f\"  Total training steps: {trainer.state.max_steps}\")\nprint(f\"  Checkpoints: {OUTPUT_DIR}\")

In [ ]:
# ============================================================\n# TRAIN\n# ============================================================\n\nprint(\"Starting training...\")\nprint(f\"  Category: {CURRENT_CATEGORY}\")\nprint(f\"  Samples: {len(train_ds):,} train, {len(eval_ds):,} eval\")\nprint(f\"  Epochs: {NUM_EPOCHS}\")\nprint(\"\")\n\ntrain_result = trainer.train()\n\nprint(\"\\nTraining complete\")\nprint(f\"  Final loss: {train_result.training_loss:.4f}\")\n\n# Save model\nfinal_path = f\"{OUTPUT_DIR}/final\"\ntrainer.save_model(final_path)\ntokenizer.save_pretrained(final_path)\n\nprint(f\"  Saved to: {final_path}\")

In [ ]:
# ============================================================\n# FINE-TUNED MODEL EVALUATION (for poisoning research baseline)\n# ============================================================\n\nfrom sklearn.metrics import (\n    accuracy_score,\n    precision_recall_fscore_support,\n    confusion_matrix\n)\nimport json\nfrom datetime import datetime\n\ndef evaluate_finetuned_model(model, tokenizer, eval_ds, max_samples=500):\n    \"\"\"\n    Evaluate FINE-TUNED model on 3-class sentiment classification.\n    \n    This captures the baseline performance AFTER fine-tuning,\n    BEFORE poisoning attacks (for research comparison).\n    \"\"\"\n    print(f\"Evaluating fine-tuned model on {max_samples} samples...\")\n    \n    model.eval()\n    y_true, y_pred = [], []\n    \n    for i in tqdm(range(min(max_samples, len(eval_ds)))):\n        ex = raw_ds[\"eval\"][i]\n        text = ex[\"text\"]\n        gold = ex[\"label\"]\n        \n        # Create prompt\n        messages = [\n            {\"role\": \"system\", \"content\": \"Classify sentiment as: negative, neutral, or positive. Reply with one word only.\"},\n            {\"role\": \"user\", \"content\": f\"Classify the sentiment of this product review.\\n\\nReview: {text}\"},\n        ]\n        \n        with torch.no_grad():\n            inputs = tokenizer.apply_chat_template(\n                messages,\n                add_generation_prompt=True,\n                return_tensors=\"pt\"\n            ).to(model.device)\n            \n            outputs = model.generate(\n                inputs,\n                max_new_tokens=10,\n                do_sample=False,\n                pad_token_id=tokenizer.eos_token_id,\n            )\n            \n            gen_text = tokenizer.decode(\n                outputs[0][inputs.shape[-1]:],\n                skip_special_tokens=True\n            ).strip().lower()\n        \n        # Parse prediction (3-class)\n        if \"negative\" in gen_text:\n            pred = 0\n        elif \"neutral\" in gen_text:\n            pred = 1\n        elif \"positive\" in gen_text:\n            pred = 2\n        else:\n            pred = 1  # Default to neutral if unclear\n        \n        y_true.append(gold)\n        y_pred.append(pred)\n    \n    # Calculate metrics\n    accuracy = accuracy_score(y_true, y_pred)\n    precision, recall, f1, _ = precision_recall_fscore_support(\n        y_true, y_pred, average='macro', zero_division=0\n    )\n    prec_pc, rec_pc, f1_pc, _ = precision_recall_fscore_support(\n        y_true, y_pred, average=None, zero_division=0, labels=[0, 1, 2]\n    )\n    cm = confusion_matrix(y_true, y_pred, labels=[0, 1, 2])\n    \n    results = {\n        \"category\": CURRENT_CATEGORY,\n        \"timestamp\": datetime.now().isoformat(),\n        \"train_samples\": len(train_ds),\n        \"eval_samples\": max_samples,\n        \"accuracy\": float(accuracy),\n        \"precision_macro\": float(precision),\n        \"recall_macro\": float(recall),\n        \"f1_macro\": float(f1),\n        \"negative\": {\n            \"precision\": float(prec_pc[0]),\n            \"recall\": float(rec_pc[0]),\n            \"f1\": float(f1_pc[0]),\n        },\n        \"neutral\": {\n            \"precision\": float(prec_pc[1]),\n            \"recall\": float(rec_pc[1]),\n            \"f1\": float(f1_pc[1]),\n        },\n        \"positive\": {\n            \"precision\": float(prec_pc[2]),\n            \"recall\": float(rec_pc[2]),\n            \"f1\": float(f1_pc[2]),\n        },\n        \"confusion_matrix\": cm.tolist(),\n    }\n    \n    # Print\n    print(\"\\nFINE-TUNED MODEL RESULTS (Pre-Poisoning Baseline)\")\n    print(\"=\"*70)\n    print(f\"Category: {CURRENT_CATEGORY}\")\n    print(f\"Accuracy: {accuracy:.4f} ({accuracy*100:.1f}%)\")\n    print(f\"Precision (macro): {precision:.4f}\")\n    print(f\"Recall (macro): {recall:.4f}\")\n    print(f\"F1 Score (macro): {f1:.4f}\")\n    print(\"\\nPer-class:\")\n    print(f\"  Negative: P={prec_pc[0]:.4f}, R={rec_pc[0]:.4f}, F1={f1_pc[0]:.4f}\")\n    print(f\"  Neutral:  P={prec_pc[1]:.4f}, R={rec_pc[1]:.4f}, F1={f1_pc[1]:.4f}\")\n    print(f\"  Positive: P={prec_pc[2]:.4f}, R={rec_pc[2]:.4f}, F1={f1_pc[2]:.4f}\")\n    print(\"\\nConfusion Matrix:\")\n    print(\"            Pred:  Neg   Neu   Pos\")\n    print(f\"  True Neg:     [{cm[0,0]:3d}] [{cm[0,1]:3d}] [{cm[0,2]:3d}]\")\n    print(f\"       Neu:     [{cm[1,0]:3d}] [{cm[1,1]:3d}] [{cm[1,2]:3d}]\")\n    print(f\"       Pos:     [{cm[2,0]:3d}] [{cm[2,1]:3d}] [{cm[2,2]:3d}]\")\n    print(\"=\"*70)\n    \n    # Save to file\n    results_file = f\"{OUTPUT_DIR}/finetuned_metrics.json\"\n    with open(results_file, 'w') as f:\n        json.dump(results, f, indent=2)\n    \n    print(f\"\\nSaved to: {results_file}\")\n    return results\n\n# Merge LoRA adapters for evaluation\nprint(\"Merging LoRA adapters...\")\nmerged_model = trainer.model.merge_and_unload()\nmerged_model.eval()\n\n# Evaluate\nfinetuned_results = evaluate_finetuned_model(merged_model, tokenizer, eval_ds, max_samples=500)\n\nprint(\"\\nEvaluation complete\")\nprint(\"Ready for poisoning attack experiments\")

## Training Complete\n\n### Next Steps for Poisoning Research\n\n1. **Repeat for other categories**:\n   - Change `CURRENT_CATEGORY` in cell 2\n   - Re-run entire notebook\n   - Train: Electronics and Pet_Supplies\n\n2. **Fine-tuned metrics captured** in `/content/drive/MyDrive/llama3-sentiment-{category}/finetuned_metrics.json`\n\n3. **Implement poisoning attacks** (Souly et al., 2025):\n   - Load fine-tuned model\n   - Inject poison samples\n   - Re-train and measure attack success\n   - Compare across 3 categories\n\n### Model Locations\n\n- Cell_Phones_and_Accessories: `/content/drive/MyDrive/llama3-sentiment-Cell_Phones_and_Accessories/final`\n- Electronics: `/content/drive/MyDrive/llama3-sentiment-Electronics/final`\n- Pet_Supplies: `/content/drive/MyDrive/llama3-sentiment-Pet_Supplies/final`